### Model Training

#### 1.1 Import Data and Required Packages
##### Importing Pandas, Numpy, Matplotlib, Seaborn and Warings Library.

In [5]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from sklearn.neural_network import MLPRegressor
import warnings

#### Import the CSV Data as Pandas DataFrame

In [6]:
df = pd.read_csv('../../data/preproccedData/Augmented_PreProccedNutrationParameters.csv')

In [7]:
df.head()

,Age,Gender,Height,Weight,Carbohydrate_Consumption,Protein_Intake,Fat_Intake,Regularity_of_Meals,Portion_Control,Hydration,Caloric_Balance,Sugar_Consumption,BMI,DiabetesRisk,RiskLevel,NutritionRisk
0,24,1,152.147228,46.315314,1.995185,1.001281,0.915818,0.921180,1.084318,2.074739,2102.955635,1.023784,17.998467,30.774236,1,65.170595
1,24,1,150.339006,50.976000,0.989616,2.020910,0.922541,0.000000,1.047479,1.065013,2102.960320,1.010403,20.263407,32.703477,1,32.574707
2,28,0,139.384921,44.483725,2.989717,3.045920,0.913793,0.000000,2.136546,2.104769,2102.947479,1.024581,20.575243,28.756909,0,16.287054
3,24,0,148.504330,51.890592,1.987570,0.994205,0.911641,0.913759,3.219068,5.253023,1375.782749,2.020694,21.127510,37.049640,1,32.571449
4,22,0,140.311087,42.626306,1.982347,1.024848,0.924424,0.925901,2.158257,3.161932,2102.954222,3.024960,19.458831,24.567752,0,16.278099


#### Preparing X and Y variables

In [9]:
X = df.drop(columns=['NutritionRisk'],axis=1)

In [10]:
X.head()

,Age,Gender,Height,Weight,Carbohydrate_Consumption,Protein_Intake,Fat_Intake,Regularity_of_Meals,Portion_Control,Hydration,Caloric_Balance,Sugar_Consumption,BMI,DiabetesRisk,RiskLevel
0,24,1,152.147228,46.315314,1.995185,1.001281,0.915818,0.921180,1.084318,2.074739,2102.955635,1.023784,17.998467,30.774236,1
1,24,1,150.339006,50.976000,0.989616,2.020910,0.922541,0.000000,1.047479,1.065013,2102.960320,1.010403,20.263407,32.703477,1
2,28,0,139.384921,44.483725,2.989717,3.045920,0.913793,0.000000,2.136546,2.104769,2102.947479,1.024581,20.575243,28.756909,0
3,24,0,148.504330,51.890592,1.987570,0.994205,0.911641,0.913759,3.219068,5.253023,1375.782749,2.020694,21.127510,37.049640,1
4,22,0,140.311087,42.626306,1.982347,1.024848,0.924424,0.925901,2.158257,3.161932,2102.954222,3.024960,19.458831,24.567752,0


In [11]:
y = df['NutritionRisk']

In [12]:
y

0       65.170595
1       32.574707
2       16.287054
3       32.571449
4       16.278099
          ...    
1057    31.015505
1058    62.009145
1059    62.013591
1060    46.511077
1061    46.504442
Name: NutritionRisk, Length: 1062, dtype: float64

In [13]:
num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
         ("StandardScaler", numeric_transformer, num_features),        
    ]
)

In [14]:
X = preprocessor.fit_transform(X)

In [15]:
df

,Age,Gender,Height,Weight,Carbohydrate_Consumption,Protein_Intake,Fat_Intake,Regularity_of_Meals,Portion_Control,Hydration,Caloric_Balance,Sugar_Consumption,BMI,DiabetesRisk,RiskLevel,NutritionRisk
0,24,1,152.147228,46.315314,1.995185,1.001281,0.915818,0.921180,1.084318,2.074739,2102.955635,1.023784,17.998467,30.774236,1,65.170595
1,24,1,150.339006,50.976000,0.989616,2.020910,0.922541,0.000000,1.047479,1.065013,2102.960320,1.010403,20.263407,32.703477,1,32.574707
2,28,0,139.384921,44.483725,2.989717,3.045920,0.913793,0.000000,2.136546,2.104769,2102.947479,1.024581,20.575243,28.756909,0,16.287054
3,24,0,148.504330,51.890592,1.987570,0.994205,0.911641,0.913759,3.219068,5.253023,1375.782749,2.020694,21.127510,37.049640,1,32.571449
4,22,0,140.311087,42.626306,1.982347,1.024848,0.924424,0.925901,2.158257,3.161932,2102.954222,3.024960,19.458831,24.567752,0,16.278099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1057,45,1,156.080607,92.430012,2.075341,3.110615,0.916742,0.000000,1.073633,6.069583,1726.270601,3.933631,36.421236,75.514135,2,31.015505
1058,48,1,162.272722,106.508776,4.099804,2.069778,0.915941,0.974791,4.278174,2.015700,3093.467516,1.952974,38.823889,72.364149,2,62.009145
1059,48,0,165.374042,104.491660,5.131436,3.108396,0.913967,0.972581,4.303791,2.019146,2824.200356,2.939429,36.661739,66.542625,1,62.013591
1060,42,1,183.984194,85.689088,5.129785,3.116705,0.923195,0.987394,3.226114,3.041969,2392.980724,1.960908,24.298784,52.574598,1,46.511077


In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape, X_test.shape

((849, 15), (213, 15))

In [20]:
df.head()

,Age,Gender,Height,Weight,Carbohydrate_Consumption,Protein_Intake,Fat_Intake,Regularity_of_Meals,Portion_Control,Hydration,Caloric_Balance,Sugar_Consumption,BMI,DiabetesRisk,RiskLevel,NutritionRisk
0,24,1,152.147228,46.315314,1.995185,1.001281,0.915818,0.921180,1.084318,2.074739,2102.955635,1.023784,17.998467,30.774236,1,65.170595
1,24,1,150.339006,50.976000,0.989616,2.020910,0.922541,0.000000,1.047479,1.065013,2102.960320,1.010403,20.263407,32.703477,1,32.574707
2,28,0,139.384921,44.483725,2.989717,3.045920,0.913793,0.000000,2.136546,2.104769,2102.947479,1.024581,20.575243,28.756909,0,16.287054
3,24,0,148.504330,51.890592,1.987570,0.994205,0.911641,0.913759,3.219068,5.253023,1375.782749,2.020694,21.127510,37.049640,1,32.571449
4,22,0,140.311087,42.626306,1.982347,1.024848,0.924424,0.925901,2.158257,3.161932,2102.954222,3.024960,19.458831,24.567752,0,16.278099


In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

# Define the preprocessing for numerical and categorical features
numerical_features = ['Age', 'Height', 'Weight', 'Carbohydrate_Consumption', 'Protein_Intake', 'Fat_Intake',]
categorical_features = ['Gender', 'Family_History', 'Thirst']

# Create a numerical transformer (standardize and impute missing values)
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),  # Impute missing values with the median
    ('scaler', StandardScaler())  # Scale numerical features
])

# Create a categorical transformer (encode and impute missing values)
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Impute missing values with the most frequent category
    ('encoder', OneHotEncoder(handle_unknown='ignore'))  # Encode categorical variables
])

# Combine both transformations in a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

# Apply the transformations
X_transformed = preprocessor.fit_transform(df)


In [21]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [22]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Assuming evaluate_model() function is defined as:
def evaluate_model(y_true, y_pred):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    return mae, rmse, r2

models = {
    "MLPRegressor": MLPRegressor(max_iter=500),  # Increasing max_iter for convergence
    "Linear Regression": LinearRegression(),
    "Lasso": Lasso(),
    "Ridge": Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree": DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor(),
}

model_list = []
r2_list_train = []
r2_list_test = []

# Split the data (assuming X and y are your features and target variables)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

for model_name, model in models.items():
    print(f"Training {model_name}...")

    # Train the model
    model.fit(X_train, y_train)
    
    # Make predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Evaluate Train and Test dataset
    model_train_mae, model_train_rmse, model_train_r2 = evaluate_model(y_train, y_train_pred)
    model_test_mae, model_test_rmse, model_test_r2 = evaluate_model(y_test, y_test_pred)
    
    # Print model performance
    print(f"Model performance for {model_name} (Training set):")
    print(f"- RMSE: {model_train_rmse:.4f}")
    print(f"- MAE: {model_train_mae:.4f}")
    print(f"- R2: {model_train_r2:.4f}")
    print("-" * 40)
    
    print(f"Model performance for {model_name} (Test set):")
    print(f"- RMSE: {model_test_rmse:.4f}")
    print(f"- MAE: {model_test_mae:.4f}")
    print(f"- R2: {model_test_r2:.4f}")
    print("=" * 40)
    
    # Store model performance
    model_list.append(model_name)
    r2_list_train.append(model_train_r2)
    r2_list_test.append(model_test_r2)

# Create a DataFrame for comparison (optional)
import pandas as pd
model_performance_df = pd.DataFrame({
    'Model': model_list,
    'Train R2': r2_list_train,
    'Test R2': r2_list_test
})

print(model_performance_df)


Training MLPRegressor...


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_erro

Model performance for MLPRegressor (Training set):
- RMSE: 7.6759
- MAE: 5.8717
- R2: 0.8089
----------------------------------------
Model performance for MLPRegressor (Test set):
- RMSE: 8.7667
- MAE: 6.4884
- R2: 0.7529
Training Linear Regression...
Model performance for Linear Regression (Training set):
- RMSE: 12.3678
- MAE: 9.6819
- R2: 0.5038
----------------------------------------
Model performance for Linear Regression (Test set):
- RMSE: 12.8734
- MAE: 9.9799
- R2: 0.4673
Training Lasso...
Model performance for Lasso (Training set):
- RMSE: 12.8115
- MAE: 10.1367
- R2: 0.4675
----------------------------------------
Model performance for Lasso (Test set):
- RMSE: 13.3429
- MAE: 10.6055
- R2: 0.4277
Training Ridge...
Model performance for Ridge (Training set):
- RMSE: 12.3678
- MAE: 9.6809
- R2: 0.5038
----------------------------------------
Model performance for Ridge (Test set):
- RMSE: 12.8716
- MAE: 9.9770
- R2: 0.4674
Training K-Neighbors Regressor...
Model performance 

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Model performance for Random Forest Regressor (Training set):
- RMSE: 2.1594
- MAE: 1.3294
- R2: 0.9849
----------------------------------------
Model performance for Random Forest Regressor (Test set):
- RMSE: 5.3057
- MAE: 3.0993
- R2: 0.9095
Training XGBRegressor...


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Model performance for XGBRegressor (Training set):
- RMSE: 0.1141
- MAE: 0.0794
- R2: 1.0000
----------------------------------------
Model performance for XGBRegressor (Test set):
- RMSE: 4.5988
- MAE: 2.5411
- R2: 0.9320
Training CatBoosting Regressor...


c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


Model performance for CatBoosting Regressor (Training set):
- RMSE: 0.9173
- MAE: 0.6705
- R2: 0.9973
----------------------------------------
Model performance for CatBoosting Regressor (Test set):
- RMSE: 3.8266
- MAE: 2.3552
- R2: 0.9529
Training AdaBoost Regressor...
Model performance for AdaBoost Regressor (Training set):
- RMSE: 8.9728
- MAE: 7.5815
- R2: 0.7388
----------------------------------------
Model performance for AdaBoost Regressor (Test set):
- RMSE: 10.3904
- MAE: 8.7921
- R2: 0.6530
                     Model  Train R2   Test R2
0             MLPRegressor  0.808855  0.752945
1        Linear Regression  0.503762  0.467272
2                    Lasso  0.467519  0.427699
3                    Ridge  0.503760  0.467420
4    K-Neighbors Regressor  0.848036  0.768311
5            Decision Tree  1.000000  0.831062
6  Random Forest Regressor  0.984872  0.909510
7             XGBRegressor  0.999958  0.932016
8    CatBoosting Regressor  0.997270  0.952931
9       AdaBoost Regre

c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
c:\ProgramData\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
